## RagAssistant

#### Dependendecies

In [4]:
! pip install swarmauri[full]==0.1.98

   ---------------------------------------- 0.0/224.2 kB ? eta -:--:--
   - -------------------------------------- 10.2/224.2 kB ? eta -:--:--
   --- ----------------------------------- 20.5/224.2 kB 330.3 kB/s eta 0:00:01
   ----- --------------------------------- 30.7/224.2 kB 325.1 kB/s eta 0:00:01
   ---------- ---------------------------- 61.4/224.2 kB 469.7 kB/s eta 0:00:01
   -------------------- ----------------- 122.9/224.2 kB 722.1 kB/s eta 0:00:01
   ---------------------------------------- 224.2/224.2 kB 1.1 MB/s eta 0:00:00
  Attempting uninstall: swarmauri
    Found existing installation: swarmauri 0.1.97
    Uninstalling swarmauri-0.1.97:
      Successfully uninstalled swarmauri-0.1.97


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
swarmauri-developer 0.0.8 requires swarmauri[full]==0.1.80, but you have swarmauri 0.1.98 which is incompatible.


In [1]:
from datetime import datetime
import sqlite3
import gradio as gr
import pandas as pd
import json
from typing import List, Dict
from swarmauri.standard.models.concrete.OpenAIModel import OpenAIModel
from swarmauri.standard.models.concrete.GroqModel import GroqModel
from swarmauri.standard.models.concrete.MistralModel import MistralModel
from swarmauri.standard.vector_stores.concrete.TFIDFVectorStore import TFIDFVectorStore
from swarmauri.standard.vector_stores.concrete.Doc2VecVectorStore import Doc2VecVectorStore
from swarmauri.standard.vector_stores.concrete.MLMVectorStore import MLMVectorStore
from swarmauri.standard.conversations.concrete.SessionCacheConversation import SessionCacheConversation
from swarmauri.standard.agents.concrete.RagAgent import RagAgent
from typing import Any, Optional, Union, Dict
from swarmauri.core.messages import IMessage
from swarmauri.core.models.IModel import IModel
from swarmauri.standard.conversations.base.SystemContextBase import SystemContextBase
from swarmauri.standard.agents.base.VectorStoreAgentBase import VectorStoreAgentBase
from swarmauri.standard.vector_stores.base.VectorDocumentStoreRetrieveBase import VectorDocumentStoreRetrieveBase
from swarmauri.standard.documents.concrete.Document import Document
from swarmauri.standard.documents.concrete.EmbeddedDocument import EmbeddedDocument
from swarmauri.standard.messages.concrete import (HumanMessage, 
                                                  SystemMessage,
                                                  AgentMessage)
from swarmauri.standard.utils.load_documents_from_json import load_documents_from_json_file

C:\Users\bigman\anaconda3\envs\swarmauri\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


#### Project Class

In [2]:
head="""<link rel="icon" href="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAACAAAAAgCAYAAABzenr0AAAAAXNSR0IArs4c6QAABWFJREFUWEe1V2tsFFUU/s7M7KO77Ra2LRisUArasLjbLqVdaRfQkACCNISgUQJCiCHGfyaYEIMSfIQY/K3GEAUkqNHwUJ6JEoht7ZZuu92uaCktILUJpSy0233PzNU7SClNu9MWvX8ms+ec7/vOuefMvUsYxypxu2dYgHRLS8utcbhjrttdIMJiaG+p69HzJz0Hl8dTKMLYwRiIMeW11ov1hzLFlFZUbSQS93EfNZGaGwz6ujP56wpwOCoeM2abrhOREUBCluXKNv+vbaOBOssXOSVJagRgZgxJOZaaFQr5bj6SAB7sqvS+KoA+J4KJqawucLHWOxpoWaW3joiqOLmqsm3BptqDehXWrcB9gPlub6lkwGcEeAC8xxgjgmDhdgY1xp9E9A7AGmRZfj3U7AvqkWsx43G67+NaWLVeFMVvAIhjxCmKorwcbKr/fry4ExJQ5vFeI9CsTOCMsauBxtri/1yAy+WyCmZbhIgyimaMsd7ueHZPj1/bFr01gQqUG9yerBhAUmZQJrfIcQv8/rQe+YR7oKzS20ZET+tsQSjQWOscD7muAN7p5wOBXAuZp0GQ8g4fPrgjMhCpUWQZ4XAYf3Xf+8Y8XlgIu90OUZJgs9mOv7Jh00dGSehTTOgtLy4eICI2liBtC862tlrzJOtWEK0gYDZAFkbMAgYbANPwaWGMIRpPIJVKI3zrpjZH9vzpMBoNsGaZ+SgO5+LESRAGiFEMYDEGXCWiM32pyJcrSkuj1NDaUSgahJ8JeCpT2RRFwZ2BCG6F7yIWT2iuT0yza88bvWHtackyo8A+BVNtORDFsSZ1iKU9lZKXUdOlK98C9NJo5IqiIhKNItw/gP7IIFT1XiVzrBZMy5uKntbvtIxnuF7EzdthRKL3Gl8QCLk52bDn2pCTbYUoCKPmxsC+oabfO/t5qXlpeZaxRBLRWFwDG4zFwX/nyyBJGmC+PRdmkwn1F85i3ewTmu3otTVYtGQ5Eskk+sL9muC0LGs2LjDbmoUciwXZlixkmU1adbStYrhLvlBH/HLXdXMilYaqqg8pNRoMyM2xYootR8uaB3FBZ06dgDfvDJ51GTT/88E06u+swvKVq4Z8eAJ3ByLoj0SRSj88kYIgwGw0oGROUYIaQx117V1/VimqAk5oNhm1veSE/H14U8XjcXy1fz8W5Aewrpr35oN1pC6JQNiNjZu3wGw2Dxm4YC4gMhhDLJFAIpnS3kVBREnxrHpq/qOrRlXZsUznAgf5LRTCoQMHcPt2X6ZeRX5BATZt3oJ5DsfIiRgZx5iqrtVmpulS5xsA7QWYdroNX52dV3Dyhx/RFmzNSDzS6Corwws1NSgqmj2aEN6t2xc65nw6NLQNbV3TRUFdCyInAWZFUbs+2L3r3e4bNx6u9YRkADNnFiV37t71PhgVCwL/FqAtLUePP+N0aheVjGfB1g1L/YudxgXi6FOkK0VRgV+CqZYvvr6wYCznjAKSDav3GiVhuy5TBgdZVT82VJ58a1ICButWlVlNQjOf5smJYCwWVcqtS0+3TEoAD0o1rD5nkITnJiNAVtg5g+fEskyxupm5K6sXgoSGDNewsfAVQsrT7PP5H0kADy6tqN4jCMKOiVRBYezDYGPtTr0Y3Qr8CyCWVniPCgKt0QPkdsbYsUBj7XoAip7/eAXA4XAYXfMK9lU8KW1kY9wLiTHm75APtF8b3Ob/P65kPJven55fb7MIu0xGmv9gOhhLy2iLJtnuqUtPHtHLerh93BUYCdp1emXJJ8eS5xlAb9ZkLSlcferyRIjv+05aAAco8yz28UM94Kvl/5YmtR5JgKuy+m0wsODFuj2TYv8n6G+wcRzTJW9piwAAAABJRU5ErkJggg==" type="image/png">"""

In [11]:
class RagAssistant:
    def __init__(self, api_key: str = "", db_path='prompt_responses.db', 
                 vector_store_path=None, model_name = "openai_gpt-4-0125-preview"):
        print('Initializing... this will take a moment.')
        self.api_key = api_key
        self.db_path = db_path
        self.conversation = SessionCacheConversation(max_size=2, system_message_content="")
        self.allowed_models =sorted(['groq_llama3-8b-8192',
                            'groq_llama3-70b-8192',
                            'groq_mixtral-8x7b-32768',
                            'groq_gemma-7b-it',
                            'mistral_mistral-medium-latest',
                            'mistral_mistral-small-latest',
                            'mistral_open-mixtral-8x22b',
                            'mistral_open-mixtral-8x7b',
                            'mistral_open-mistral-7b',
                            'mistral_mistral-large-latest',
                            'openai_gpt-3.5-turbo',
                            'openai_gpt-3.5-turbo-16k',
                            'openai_gpt-4-0125-preview'
                              ])
            
        self.model = None
        self.retrieval_table = []
        self.document_table = []
        self.long_term_memory_df = None
        self.last_recall_df = None
        self.agent = self.initialize_agent()
        self.set_model(model_name)
        self.css = """
#chat-dialogue-container {
    min-height: 54vh !important;
}

#document-table-container {
    min-height: 80vh !important;
}

footer {
    display: none !important;
}
"""
        self.setup_gradio_interface()
        
    def initialize_agent(self):
        VS = Doc2VecVectorStore()
        agent = RagAgent(name="Rag", model=self.model, conversation=self.conversation, vector_store=VS)
        return agent

    def set_model(self, provider_model_choice: str):
        if provider_model_choice in self.allowed_models:
            provider, model_name = provider_model_choice.split('_')
            if provider == 'groq':
                self.model = GroqModel(api_key=self.api_key, model_name=model_name)
                
            if provider == 'mistral':
                self.model = MistralModel(api_key=self.api_key, model_name=model_name)
                
            if provider == 'openai':
                self.model = OpenAIModel(api_key=self.api_key, model_name=model_name)
                
            self.agent.model = self.model
                
        else:
            raise ValueError(f"Model name '{model_name}' is not supported. Choose from {self.allowed_models}")

    def change_vectorizer(self, vectorizer: str):
        if vectorizer == 'Doc2Vec':
            self.agent.vector_store = Doc2VecVectorStore()
        if vectorizer == 'MLM':
            self.agent.vector_store = MLMVectorStore()
        else:
            self.agent.vector_store = TFIDFVectorStore()
            
    
    def load_and_filter_json(self, file_info):
        # Load JSON file using json library
        try:
            documents = load_documents_from_json_file(file_info.name)
            self.agent.vector_store.documents = []
            self.agent.vector_store.add_documents(documents)
            self.long_term_memory_df = self.preprocess_documents(documents)
            return self.long_term_memory_df
        except json.JSONDecodeError:
            return "Invalid JSON file. Please check the file and try again."

    def preprocess_documents(self, documents):
        try:
            docs = [d.to_dict() for d in documents]
            for d in docs:
                metadata = d['metadata']
                for each in metadata:
                    d[each] = metadata[each]
                del d['metadata']
                del d['type']
                del d['embedding']
            df = pd.DataFrame.from_dict(docs)
            return df
        except Exception as e:
            print(f"postprocess_documents: {e}")

    
    def sql_log(self, conversation_id, model_name, prompt, response, start_datetime, end_datetime):
        try:
            duration = (end_datetime - start_datetime).total_seconds()
            start_datetime = start_datetime.isoformat()
            end_datetime = end_datetime.isoformat()
            conversation_id = str(conversation_id)
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            cursor.execute('''CREATE TABLE IF NOT EXISTS conversations
                            (id INTEGER PRIMARY KEY AUTOINCREMENT, 
                            conversation_id TEXT, 
                            model_name TEXT, 
                            prompt TEXT, 
                            response TEXT, 
                            start_datetime TEXT, 
                            end_datetime TEXT,
                            duration NUMERIC)''')
            cursor.execute('''INSERT INTO conversations (
                            conversation_id, 
                            model_name, 
                            prompt, 
                            response, 
                            start_datetime, 
                            end_datetime,
                            duration) VALUES (?, ?, ?, ?, ?, ?, ?)''', 
                           (conversation_id, 
                            model_name, 
                            prompt, 
                            response, 
                            start_datetime, 
                            end_datetime, 
                            duration))
            conn.commit()
            conn.close()
        except:
            raise
    
    def save_df(self, df):
        documents = self.dataframe_to_documents(df)
        self.agent.vector_store.documents = []
        self.agent.vector_store.add_documents(documents)
        self.long_term_memory_df = self.preprocess_documents(documents)
        return self.long_term_memory_df

    def dataframe_to_documents(self, df):
        documents = []
        for index, row in df.iterrows():
            row_dict = row.to_dict() 
            id = row_dict.pop("id", "")
            content = row_dict.pop("content", "")
            metadata = row_dict  # remaining data becomes metadata
        
            # Convert the row to dictionary and create a DocumentBase instance
            
            document = EmbeddedDocument(id=id, content=content, metadata=metadata)
            documents.append(document)
        return documents
        
    def clear_chat(self):
        # We could clear, but if we create a new instance, we get a new conversation id
        max_size = self.agent.conversation.max_size
        session_cache_size = self.agent.conversation.session_cache_max_size
        self.conversation = SessionCacheConversation(max_size=max_size, 
                                                     system_message_content="", 
                                                     session_cache_max_size=session_cache_size)
        self.agent.conversation = self.conversation
        return "", [], []    
        
    async def chatbot_function(self, 
                         message, 
                         history, 
                         api_key: str = None, 
                         model_name: str = None, 
                         top_k: int = 5, 
                         temperature: int = 1, 
                         max_tokens: int = 256,
                         conversation_size: int = 2,
                         session_cache_size: int = 2):
        try:
            start_datetime = datetime.now()
            if self.agent.vector_store.document_count() == 0:
                return "", [], [(message, "⚠️ Add Documents First")]
            else:

                
                
                # Set additional parameters
                self.api_key = api_key
                self.set_model(model_name)
                #print(self.model, self.model.model_name, self.api_key)

                # Set Conversation Size
                self.agent.conversation.max_size = conversation_size
                self.agent.conversation.session_cache_max_size = session_cache_size
                
                # Predict
                response = self.agent.exec(message, top_k=top_k, model_kwargs={'temperature': temperature, 'max_tokens': max_tokens})
    
                # Update Retrieval Document Table
                self.last_recall_df = self.preprocess_documents(self.agent.last_similar_documents)
                
                # Get History
                history = [each['content'] for each in self.agent.conversation.session_to_dict()]
                history = [(history[i], history[i+1]) for i in range(0, len(history), 2)]

                # SQL Log
                end_datetime = datetime.now()
                self.sql_log(self.agent.conversation.id, model_name, message, response, start_datetime, end_datetime)
                
                return "", self.last_recall_df, history
        except Exception as e:
            gr.Error(f"{e}")
            self.agent.conversation._history.pop(0)
            print(f"chatbot_function error: {e}")
            return "", [], history
    

        
    def setup_gradio_interface(self):
        with gr.Blocks(css = self.css) as self.retrieval_table:
            with gr.Row():
                self.retrieval_table = gr.Dataframe(interactive=False, wrap=True, line_breaks=True, elem_id="document-table-container", height="800")

        
        with gr.Blocks(css = self.css) as self.chat:
            with gr.Row():
                self.chat_history = gr.Chatbot(label="Chat History", 
                                           layout="panel", 
                                           elem_id="chat-dialogue-container", 
                                           container=True, 
                                           show_copy_button=True,
                                           height="70vh")
            with gr.Row():
                self.input_box = gr.Textbox(label="Type here:", scale=6)
                self.send_button = gr.Button("Send", scale=1)
                self.clear_button = gr.Button("Clear", scale=1)
                
            with gr.Accordion("See Details", open=False):
                self.additional_inputs = [
                    gr.Textbox(label="API Key", value=self.api_key or "Enter your API Key"),
                    gr.Dropdown(self.allowed_models, 
                                value="openai_gpt-3.5-turbo", 
                                label="Model",
                                info="Select openai model"),
                    gr.Slider(label="Top K", value=10, minimum=0, maximum=100, step=5, interactive=True),
                    gr.Slider(label="Temperature", value=1, minimum=0.0, maximum=1.5, step=0.05, interactive=True),
                    gr.Slider(label="Max new tokens", value=256, minimum=256, maximum=4096, step=64, interactive=True),
                    gr.Slider(label="Conversation size", value=12, minimum=2, maximum=36, step=2, interactive=True),
                    gr.Slider(label="Session Cache size", value=12, minimum=2, maximum=36, step=2, interactive=True)
                ]
    
    
            submit_inputs = [self.input_box, self.chat_history]
            submit_inputs.extend(self.additional_inputs)
            # Function to handle sending messages
            self.send_button.click(
                self.chatbot_function, 
                inputs=submit_inputs, 
                outputs=[self.input_box, self.retrieval_table, self.chat_history]
            )
        
            # Function to handle clearing the chat
            self.clear_button.click(
                self.clear_chat, 
                inputs=[], 
                outputs=[self.input_box, self.retrieval_table, self.chat_history]
            )

        
        with gr.Blocks(css = self.css) as self.document_table:
            with gr.Row():
                self.file = gr.File(label="Upload JSON File")
                self.vectorizer = gr.Dropdown(choices=["Doc2Vec", "TFIDF", "MLM"], value="Doc2Vec", label="Select vectorizer")
                self.load_button = gr.Button("load")
            with gr.Row():
                self.data_frame = gr.Dataframe(interactive=True, wrap=True, line_breaks=True, elem_id="document-table-container", height="700")
            with gr.Row():
                self.save_button = gr.Button("save")
                
            self.vectorizer.change(self.change_vectorizer, inputs=[self.vectorizer], outputs=self.data_frame)
            self.load_button.click(self.load_and_filter_json, inputs=[self.file], outputs=self.data_frame)
            self.save_button.click(self.save_df, inputs=[self.data_frame])

        with gr.Blocks(css = self.css, title="Swarmauri Rag Agent", head=head) as self.app:
            gr.TabbedInterface(interface_list=[self.chat, self.retrieval_table, self.document_table], 
                                      tab_names=["chat", "retrieval", "documents"])

    
    def launch(self):
        self.app.launch(share = False,
                        #server_name = "192.168.1.139",
                        favicon_path = "./favicon-32x32.png")

if __name__ == "__main__":
    rag_assistant = RagAssistant(db_path = "E:\\swarmauri_github\\prompt_responses.db")
    rag_assistant.launch()

Initializing... this will take a moment.
Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


chatbot_function error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `mixtral-8x7b-32768` in organization `org_01hw3n80m2fsw8vpd7dyg9tp5f` on tokens per minute (TPM): Limit 4500, Used 0, Requested ~8307. Please try again in 50.76s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


Traceback (most recent call last):
  File "C:\Users\bigman\anaconda3\envs\swarmauri\lib\site-packages\gradio\queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
  File "C:\Users\bigman\anaconda3\envs\swarmauri\lib\site-packages\gradio\route_utils.py", line 261, in call_process_api
    output = await app.get_blocks().process_api(
  File "C:\Users\bigman\anaconda3\envs\swarmauri\lib\site-packages\gradio\blocks.py", line 1786, in process_api
    result = await self.call_function(
  File "C:\Users\bigman\anaconda3\envs\swarmauri\lib\site-packages\gradio\blocks.py", line 1338, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "C:\Users\bigman\anaconda3\envs\swarmauri\lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "C:\Users\bigman\anaconda3\envs\swarmauri\lib\site-packages\anyio\_backends\_asyncio.py", line 2144, in run_sync_in_worker_thread
    

chatbot_function error: database is locked
chatbot_function error: database is locked
chatbot_function error: database is locked
chatbot_function error: database is locked
chatbot_function error: database is locked
chatbot_function error: database is locked
chatbot_function error: database is locked
chatbot_function error: database is locked
chatbot_function error: database is locked
chatbot_function error: database is locked
chatbot_function error: database is locked
chatbot_function error: database is locked
chatbot_function error: database is locked
chatbot_function error: database is locked
chatbot_function error: database is locked
chatbot_function error: database is locked
chatbot_function error: database is locked
chatbot_function error: database is locked
chatbot_function error: database is locked
chatbot_function error: database is locked
chatbot_function error: database is locked
chatbot_function error: database is locked
chatbot_function error: database is locked
chatbot_fun

In [15]:
rag_assistant.agent.conversation.history
len(rag_assistant.agent.conversation.history)

ValueError: System context has not been set.

In [13]:
rag_assistant.agent.conversation.history

[SystemMessage(role='system'), HumanMessage(role='user')]

In [14]:
rag_assistant.agent.conversation.as_messages()[1:]

[{'content': "let's create pydantic v2 create, get, and update model schemas for standard/models/concrete/GroqModel",
  'role': 'user'}]